In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as offline
import pandas as pd 
import numpy as np
import plotly.io as pio
import glob
from scipy import stats
import os
offline.init_notebook_mode(connected=True)
from IPython.display import display
from math import ceil




In [ ]:
arr=!echo $LOGDIR
FOLDER=arr[0]

In [ ]:
#Place / at the end of path
FOLDER="/zfs/dicelab/DigitalPlant/results/July_18_22_29_10_node2052_2units/"

In [ ]:
path = FOLDER

df = [{"name": "1 Node", "df": None, "color": "#ccc"}, {"name": "2 Nodes", "df": None, "color": "#000"}, {"name": "4 Nodes", "df": None, "color": "#666"}]

# Dr. Apon wants all nodes combined into one dataframe

i = 0

for directory in os.listdir(path):
    if os.path.isdir(path + directory + "/"):
        all_files = []

        nodes = path + directory + "/"

        for node in os.listdir(nodes):
            all_files += glob.glob(nodes + node + "/" + "*.csv")

        li = []

        for filename in all_files:
            frame = pd.read_csv(filename, index_col=None, header=0)
            li.append(frame)

        frame = pd.concat(li, axis=0, ignore_index=True)

        frame['Receive Time'] = pd.to_datetime(frame['Receive Time'], unit='ms')
        frame['Send Time'] = pd.to_datetime(frame['Send Time'], unit='ms')
        frame['Latency'] = (frame['Receive Time'] - frame['Send Time']).dt.total_seconds() * 1000

        df[i]["df"] = frame

        i += 1

display(df)

In [ ]:
!cat {FOLDER}

In [ ]:
for i in range(3):
    frame = df[i]["df"]
    print(frame.groupby(['Status']).count())

In [ ]:
PERCENT_TO_DROP = .05

for i in range(3):
    frame = df[i]["df"]
    num_messages = frame.shape[0]
    #print(num_messages)
    messages_to_drop = 1350
    #print(messages_to_drop)
    # print("Original dataframe contained {} messages, with each device sending {}\nDropping {}%, or 2,700,000 messages from the front".format(len(frame), num_messages/1000, PERCENT_TO_DROP,messages_to_drop))
    frame.drop(frame[frame['Message ID'] < messages_to_drop].index, inplace=True) #0 1 2 3 4
#     origSend = frame.at[messages_to_drop, "Send Time"]
#     frame["Relative Send Time"] = frame["Send Time"] - origSend
#     frame['Relative Send Time'] = frame['Relative Send Time'].apply(lambda x: float(x.total_seconds())) 
    frame.count()
    print(frame)
    
    

In [ ]:
fig_avg_lat = go.Figure()
    
bins = np.linspace(frame["Relative Send Time"].min(), frame["Relative Send Time"].max(), 5400)
groups = frame.groupby(pd.cut(frame["Relative Send Time"], bins))

fig_avg_lat.add_trace(go.Scatter(y=groups.mean().Latency, mode='lines', name="Node1", marker_color="#000"))

fig_avg_lat.update_layout(
    title = "Average Latency (ms) vs Time (s)",
    xaxis_title = "Time (s)",
    yaxis_title = "Average Latency (ms)"
)
    
fig_avg_lat.show()

In [ ]:
for i in range(3):
    frame = df[i]["df"]   
    print(frame['Latency'].describe())


In [ ]:
fig_box = go.Figure()

fig_box.add_trace(go.Box(y=frame["Latency"], quartilemethod="inclusive", name="Node1", boxpoints=False))
    
fig_box.update_layout(
    title = "Latency vs Intermessage Time",
    xaxis_title = "Intermessage Time",
    yaxis_title = "Latency (ms)"
)    

fig_box.show()

In [ ]:
fig_cdf = go.Figure()

# for arr in df:
#     fig_cdf = go.Figure()
#     for frame in arr:

# for i in range(3):
#     fig_cdf = go.Figure()
#     for arr in df:

frame = df[0]
totalMessages = len(frame["df"])
freq = frame["df"].groupby(['Latency']).size()
rel = freq / totalMessages # Divide that by the total messages to get the relative count
cumsum = rel.cumsum() #Cumulative to get the cumualive frequencies
fig_cdf.add_trace(go.Scatter(x=cumsum.index.values, y=cumsum, name="1 Node", line=dict(color=frame["color"], width=4,
                              dash='dot')))


frame = df[1]
totalMessages = len(frame["df"])
freq = frame["df"].groupby(['Latency']).size()
rel = freq / totalMessages # Divide that by the total messages to get the relative count
cumsum = rel.cumsum() #Cumulative to get the cumualive frequencies
fig_cdf.add_trace(go.Scatter(x=cumsum.index.values, y=cumsum, name="2 Nodes", line=dict(color=frame["color"], width=4, dash='dash')))


frame = df[2]
totalMessages = len(frame["df"])
freq = frame["df"].groupby(['Latency']).size()
rel = freq / totalMessages # Divide that by the total messages to get the relative count
cumsum = rel.cumsum() #Cumulative to get the cumualive frequencies
fig_cdf.add_trace(go.Scatter(x=cumsum.index.values, y=cumsum, name="4 Nodes", line=dict(color=frame["color"], width=4)))


fig_cdf.update_layout(
    legend=dict(x=-0.1, y=1.2, orientation="h",
        font = dict(
            family = 'Computer Modern',
            size = 24,
            color = '#000'
        )
    ),
    
    plot_bgcolor = '#fff',
    
    xaxis = dict(
        range = [27, 300],
        showgrid = True,
        zeroline = True,
        showline = True,
        gridcolor = '#fff',
        title = 'Latency (ms)',
        titlefont = dict(
            family = 'Computer Modern',
            size = 30,
            color = '#000'
        ),
        showticklabels = True,
        tickfont = dict(
            family = 'Computer Modern',
            size = 30,
            color = '#000'
        )
    ),
    
    yaxis = dict(
        showgrid = True,
        zeroline = True,
        showline = True,
        gridcolor = '#ccc',
        title = 'Probability',
        titlefont = dict(
            family = 'Computer Modern',
            size = 30,
            color = '#000'
        ),
        showticklabels = True,
        tickfont = dict(
            family = 'Computer Modern',
            size = 30,
            color = '#000'
        )
    )
)

# img_bytes = pio.to_image(fig_cdf, format='svg')
# display(SVG(img_bytes))

# fig_cdf.write_image("/zfs/dicelab/DigitalPlant/results/images/test.pdf") # This saves the plot as a file
fig_cdf.show()

In [ ]:
fig_mes_lat = go.Figure()

fig_mes_lat.add_trace(go.Scatter(x=frame["Relative Send Time"], 
                                 y=frame["Latency"], 
                                 name="Node1", 
                                 line=dict(
                                     color='royalblue', 
                                     width=2
                                 )
                                )
                     )

fig_mes_lat.update_layout(
    title = "Message Latency (ms) vs Relative Send Time (s)",
    xaxis_title = "Relative Send Time (s)",
    yaxis_title = "Message Latency (ms)"
) 
    
fig_mes_lat.show()